In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from store.operations import StoringPool
from bidding.operations import create_bidding_app, place_bid, cancel_bid, accept_bid, close_bidding
from trading.operations import create_trading_app, place_trade, cancel_trade, accept_trade, close_trading
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

storeAppID = int(os.environ.get("STORE_APP_ID"))
# close_bidding(client, 693, storeAppID, creator)
print(f"storeAppID: {storeAppID}")

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

storeAppID: 3310
Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


In [4]:
# def create_dummy_asset(sender: Account, total: int, decimals: int, asset_name: str, unit_name: str):
#     txn = transaction.AssetConfigTxn(
#         sender=sender.get_address(),
#         sp=client.suggested_params(),
#         total=total,
#         decimals=decimals,
#         asset_name=asset_name,
#         unit_name=unit_name,
#         default_frozen=False,
#         strict_empty_address_check=False,
#     )
#     signed_txn = txn.sign(sender.get_private_key())

#     client.send_transaction(signed_txn)

#     response = wait_for_confirmation(client, signed_txn.get_txid())
#     assert response.asset_index is not None and response.asset_index > 0
#     return response.asset_index

# asset_id = create_dummy_asset(seller, 1, 0, "example token", "AVT")

asset_id = int(os.environ.get("TOKEN_ID"))
print(f"The token id is: {asset_id}")

The token id is: 2989


In [5]:
start_time = int(time()) + 10
end_time = start_time + 10
price = 1_000_000  # 1 Algo

"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [6]:
storeApp = StoringPool(client=client, creator=creator)
storeApp.create_app()
storeAppID = storeApp.app_id

# storeAppID = int(os.environ.get("STORE_APP_ID"))
# storeApp.app_id = storeAppID
# print(f"Store App ID: {storeAppID}")

storeAppAddress = get_application_address(storeAppID)
print(f"Store App Address: {storeAppAddress}")


Waiting for confirmation...
Transaction LHJUUURWZ4Q65KM2VSQPCJ3HL6DJNPTVWSW6K4HBWX5CVT3CRT4A confirmed in round 157431.
Store App ID: 4084
Store App address: JOUU2H7VLJX3EQHRGDY6DE3EV3ZPHHSTHLYV72VQIEB24TIKREOBWNUNQM
Waiting for confirmation...
Transaction R6L5G5WZQVS55GGO4L7MJID5WWZYD6CLRL7BV7TA3YMQFWBI4DZQ confirmed in round 157433.
Store App Address: JOUU2H7VLJX3EQHRGDY6DE3EV3ZPHHSTHLYV72VQIEB24TIKREOBWNUNQM


In [8]:

appID = create_bidding_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction TA4AMPG7RK7PODJEO2MKT6BX3UXBRYN3M7DDITWBOFWXBTSUQAPA confirmed in round 157437.
Waiting for confirmation...
Transaction LHBZYOBXR5KOKK6OWMITFL2WC7YGYG6VFYIMEEYIVADMVEGDFNKQ confirmed in round 157439.
App ID: 4087
App Address: 5RHWCMQ3FKWO3MZDBRO74MCQDE2ZNCWYOZ7NB3E4FV6WDGDP2KMWLCBHX4


Alice is setting up and funding token auction...

In [9]:
tradingAppID = create_trading_app(
    client=client,
    creator=creator,
    store_app_id=storeAppID
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction 2GMS4G2INZG53WAOJSSRSV6X2SLILMQ26X234XIS25Z6IKOG3XCA confirmed in round 157441.
Waiting for confirmation...
Transaction HSOSQLXM3JTOXDNDRV7ZENBIRWT6VYZMQ7JZOMPGALY2MSPYQ7YQ confirmed in round 157443.
App ID: 4087
App Address: 5RHWCMQ3FKWO3MZDBRO74MCQDE2ZNCWYOZ7NB3E4FV6WDGDP2KMWLCBHX4


In [10]:
storeApp.set_up(trade_app_id=tradingAppID, bid_app_id=appID, auction_app_id=appID, distribution_app_id=appID)

Waiting for confirmation...
Transaction KNE733IDH7ZRY4O2447Y2OZWE7TMQ4ESQNANACECLIEUKHQVP2IA confirmed in round 157445.


In [11]:
sellerAlgosBefore = get_balances(client, seller.get_address())[0]

print("Alice's algo balance: ", sellerAlgosBefore, " algos")

Alice's algo balance:  20077389651881  algos


In [12]:
sleep(5)
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [13]:
bidPrice = price
bidderAlgosBefore = get_balances(client, bidder.get_address())[0]
print("Carla wants to bid on token, her algo balance: ", bidderAlgosBefore, " algos")
print("Carla is placing bid for: ", bidPrice, " algos")

Carla wants to bid on token, her algo balance:  10036598957960  algos
Carla is placing bid for:  1000000  algos


In [14]:
trading_index = place_trade(client=client, app_id=tradingAppID, seller=seller, token_id=asset_id, token_amount=10, price=bidPrice, trading_index="")

print("Carla bid", trading_index)

store_app_id 4084
seller CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA opt in app 4084
Waiting for confirmation...
Transaction 233L3XE5G3UDS33J77DW3VDPWVJG5CDMVWZ2PYMD67KAKTDHBBMQ confirmed in round 157448.
Waiting for confirmation...
Transaction TWGYTZQSACTYNBNCEWH73QTMTGMCXW2INYYXACCEW2DQ3DN2IC5A confirmed in round 157450.
new address: 5LHDGX2RDYD5R22TVVJSJMW2UU5ZFMW3KX5QSHF2ZZXFXEIGR7MUKQTYYM
new private_key: BmBt4pDJ4UuW+F1ilVxjr1WKgU7Q/OcztUh7JqZOOvPqzjNfUR4H2OtTrVMkstqlO5Ky21X7CRy6zm5bkQaP2Q==
new passphrase: absorb remain illness october seminar napkin seven fuel proof inch random foot believe parrot list wolf language state picture guilt era neck oil abstract uniform
Waiting for confirmation...
Transaction AAR3SESXUASUFSZHO7ZQIUYTARSWI3VMYANX4ICUMJ7VVUSJ6WDQ confirmed in round 157452.
Waiting for confirmation...
Transaction F3CCIHCE3S7DBLEGFK4CAU56I3LA6EV2L2JRS37B7QT3P5XLU5WA confirmed in round 157454.
Waiting for confirmation...
Transaction ULHWEKACUEIEE2CDDOFZJNU

In [15]:
accept_trade(client, tradingAppID, bidder, seller.get_address(), trading_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction HLOSLC32YCTNFWD3SHRMZUZHNRL3ZKJI4E4EVNVVX5TRNLG66SYQ confirmed in round 157460.
Waiting for confirmation...
Transaction WXW6JFF34IICWIBD4VOXWO5GYTA65AQI5E5W7X6SZH6ISKWY3O7A confirmed in round 157462.


True

In [16]:
bid_index = place_bid(client=client, app_id=appID, token_id=asset_id, bidder=bidder, bid_amount=10, bid_price=bidPrice, bid_index="")

print("Carla bid", asset_id)

store_app_id 4084
new address: 2BSXPIN5EZENEQPRUENKZPXC3ZXXDV4BXEJTNYUS5QUYCHFQVZFOVPE4YM
new private_key: pxCu3t7DtoauRKHmTQaisGFsdyuZ9Jk3R0G/QcW2pjHQZXehvSZI0kHxoRqsvuLeb3HXgbkTNuKS7CmBHLCuSg==
new passphrase: belt reunion hungry digital horn payment century pattern rubber bomb extra brand sugar tape sing virus veteran broom pass brief melody pumpkin minor ability hand
Waiting for confirmation...
Transaction MDADFV4ZCCDUZCF2BYAWJEIBE2WLF7VJENDL375RWV72MNUG2U3Q confirmed in round 157464.
Waiting for confirmation...
Transaction 27T63QKMKDKH7K4AE5RVTW67GJAOS4QCQ46QGA4NIQF4U6K5KBOQ confirmed in round 157466.
Waiting for confirmation...
Transaction 2WLBETGCSLERT5VW6TVBF24N54MR7TLW5HVPX4UDETRUKKLK6XQQ confirmed in round 157468.
Waiting for confirmation...
Transaction DNQ2D2E6WJF5SWEJBYTGYH6UOLXLCCZASV5ILGSZNT3HIGCMWLVA confirmed in round 157470.
Carla bid 2989


In [17]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 1104000}


In [18]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

# cancel_bid(client, appID, bidder, bid_index)

# print("Carla canceled bid", asset_id)

In [19]:
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=10, bid_price=bidPrice, bid_index="")
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=20, bid_price=bidPrice+1_000_000, bid_index=bid_index)
# print(bid_index)
# bid_index = place_bid(client=client, app_id=appID, bidder=bidder, token_id=asset_id, bid_amount=15, bid_price=bidPrice+500_000, bid_index=bid_index)
# print(bid_index)


In [20]:
bidderAppLocalState = get_app_local_state(client, appID, bid_index)
print(bidderAppLocalState)

{b'TA': 10, b'TK_ID': 2989, b'B_ADDR': b'\xbf\xde\xc1=\xf1\x1e\xa7\xbe\xe71\x9b\xc9<+\xbb7q`\xaa\x8dsj<wVp\xdf\xb6\xef\x0c\xfbL', b'TP': 1000000}


In [21]:
accept_bid(client, appID, seller, bidder.get_address(), bid_index)

token_amount 10
price 1000000
Waiting for confirmation...
Transaction NMGISL5O4ABS2WUG5LBSSHWR5OGLOZ76MIBLPHLMA5P3CNX2O2YA confirmed in round 157472.
Waiting for confirmation...
Transaction TGMSTKBZP4AFT3VZVWFBBTN7ZVFIT6FIT6GV7RR3QJIZC4ZFCH7A confirmed in round 157474.


True

Alice is closing out the bidding contract....

In [22]:
sleep(5)

close_bidding(client, appID, creator, [asset_id])

b'assets' [2989]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction MVRR7MXTOOPH2WRQGHTT26EFAUHODIOG5W75T3OUPANOKD4AMBUQ confirmed in round 157477.


In [23]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 335500, 2989: 0}


In [24]:
bidderNftBalance = get_balances(client, bidder.get_address())[asset_id]

print("Carla's NFT balance:", bidderNftBalance, " for token ID: ", asset_id)

assert bidderNftBalance == 1

actualSellerBalances = get_balances(client, seller.get_address())
print("Alice's balances after auction: ", actualSellerBalances, " Algos")

actualBidderBalances = get_balances(client, bidder.get_address())
print("Carla's balances after auction: ", actualBidderBalances, " Algos")
# assert len(actualSellerBalances) == 2
# seller should receive the bid amount, minus the txn fee
assert actualSellerBalances[0] >= sellerAlgosBefore + bidPrice - 1_000
assert actualSellerBalances[asset_id] == 0

Carla's NFT balance: 50000000040020  for token ID:  2989


AssertionError: 

In [ ]:
storeAppState = get_app_global_state(client, 1676)
print(storeAppState)
#print(encoding.encode_address(storeAppState[b"C"]))